<a href="https://colab.research.google.com/github/IT21158186/INTERVIEW-PROCESSING-SYSTEM/blob/IT21158186-Deneth_Pinsara-Model_Train/Image_Classify_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
# Install required packages
!pip install tensorflow
!pip install kaggle

# Kaggle API token setup (Make sure your Kaggle JSON file is uploaded to Colab)
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

# Download datasets using Kaggle API
!kaggle datasets download -d ziadhanyai/fashion-six-classes
!kaggle datasets download -d itsahmad/indoor-scenes-cvpr-2019

# Extract datasets
import zipfile

with zipfile.ZipFile('/content/fashion-six-classes.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/fashion')
with zipfile.ZipFile('/content/indoor-scenes-cvpr-2019.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/indoor')

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split
import shutil
import os

# Step 1: Prepare the datasets
# Define directories
fashion_dir = '/content/fashion/Fashion'
indoor_dir = '/content/indoor/indoorCVPR_09/Images'

# Create a structured directory for Dress Classification (Formal vs Casual)
os.makedirs('/content/dress_data/formal', exist_ok=True)
os.makedirs('/content/dress_data/casual', exist_ok=True)

for category in ['Formal']:
    src = os.path.join(fashion_dir, category)
    dest = '/content/dress_data/formal'
    for img in os.listdir(src):
        shutil.copy(os.path.join(src, img), os.path.join(dest, img))

for category in ['T-Shirt', 'Hoodie', 'Accessories']:
    src = os.path.join(fashion_dir, category)
    dest = '/content/dress_data/casual'
    for img in os.listdir(src):
        shutil.copy(os.path.join(src, img), os.path.join(dest, img))

# Create a structured directory for Background Classification (Clean vs Messy)
os.makedirs('/content/background_data/clean', exist_ok=True)
os.makedirs('/content/background_data/messy', exist_ok=True)

clean_folders = ['classroom', 'computerroom', 'office']
messy_folders = [folder for folder in os.listdir(indoor_dir) if folder not in clean_folders]

for folder in clean_folders:
    src = os.path.join(indoor_dir, folder)
    dest = '/content/background_data/clean'
    for img in os.listdir(src):
        shutil.copy(os.path.join(src, img), os.path.join(dest, img))

for folder in messy_folders:
    src = os.path.join(indoor_dir, folder)
    dest = '/content/background_data/messy'
    for img in os.listdir(src):
        shutil.copy(os.path.join(src, img), os.path.join(dest, img))

# Step 2: Data Generators for Dress and Background Models
dress_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
background_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

dress_train = dress_datagen.flow_from_directory(
    '/content/dress_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

dress_val = dress_datagen.flow_from_directory(
    '/content/dress_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

background_train = background_datagen.flow_from_directory(
    '/content/background_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

background_val = background_datagen.flow_from_directory(
    '/content/background_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Step 3: Model Creation Function
def create_model():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Step 4: Train the Dress Classification Model
print("Training Dress Classification Model...")
dress_model = create_model()
dress_history = dress_model.fit(
    dress_train,
    steps_per_epoch=dress_train.samples // dress_train.batch_size,
    validation_data=dress_val,
    validation_steps=dress_val.samples // dress_val.batch_size,
    epochs=10
)

# Step 5: Train the Background Classification Model
print("Training Background Classification Model...")
background_model = create_model()
background_history = background_model.fit(
    background_train,
    steps_per_epoch=background_train.samples // background_train.batch_size,
    validation_data=background_val,
    validation_steps=background_val.samples // background_val.batch_size,
    epochs=10
)

# Both models are now trained
print("Training Complete.")


Dataset URL: https://www.kaggle.com/datasets/ziadhanyai/fashion-six-classes
License(s): apache-2.0
fashion-six-classes.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset URL: https://www.kaggle.com/datasets/itsahmad/indoor-scenes-cvpr-2019
License(s): DbCL-1.0
indoor-scenes-cvpr-2019.zip: Skipping, found more recently modified local copy (use --force to force download)
Found 2449 images belonging to 2 classes.
Found 612 images belonging to 2 classes.
Found 12397 images belonging to 2 classes.
Found 3098 images belonging to 2 classes.
Training Dress Classification Model...
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 298s 4s/step - accuracy: 0.6341 - loss: 0.6579 - val_accuracy: 0.6513 - val_loss: 0.6466
Epoch 2/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5938 - loss: 0.6936 - val_accuracy: 1.0000 - val_loss: 0.4358
Epoch 3/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 262s 3s/step - accuracy: 0.6571 - l

In [30]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Step 6: Evaluate Dress Classification Model
print("Evaluating Dress Classification Model...")
dress_val.reset()  # Reset the generator for consistency
y_true_dress = dress_val.classes  # Ground truth labels
y_pred_dress = (dress_model.predict(dress_val) > 0.5).astype("int32").flatten()  # Predictions

# Calculate accuracy
dress_accuracy = accuracy_score(y_true_dress, y_pred_dress)
print(f"Dress Classification Accuracy: {dress_accuracy:.2f}")

# Print a detailed classification report
dress_class_names = list(dress_val.class_indices.keys())
print("Dress Classification Report:")
print(classification_report(y_true_dress, y_pred_dress, target_names=dress_class_names))

# Step 7: Evaluate Background Classification Model
print("Evaluating Background Classification Model...")
background_val.reset()  # Reset the generator for consistency
y_true_background = background_val.classes  # Ground truth labels
y_pred_background = (background_model.predict(background_val) > 0.5).astype("int32").flatten()  # Predictions

# Calculate accuracy
background_accuracy = accuracy_score(y_true_background, y_pred_background)
print(f"Background Classification Accuracy: {background_accuracy:.2f}")

# Print a detailed classification report
background_class_names = list(background_val.class_indices.keys())
print("Background Classification Report:")
print(classification_report(y_true_background, y_pred_background, target_names=background_class_names))


Evaluating Dress Classification Model...
20/20 ━━━━━━━━━━━━━━━━━━━━ 66s 3s/step
Dress Classification Accuracy: 0.65
Dress Classification Report:
              precision    recall  f1-score   support

      casual       0.65      1.00      0.79       400
      formal       0.00      0.00      0.00       212

    accuracy                           0.65       612
   macro avg       0.33      0.50      0.40       612
weighted avg       0.43      0.65      0.52       612

Evaluating Background Classification Model...


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


97/97 ━━━━━━━━━━━━━━━━━━━━ 243s 2s/step
Background Classification Accuracy: 0.98
Background Classification Report:
              precision    recall  f1-score   support

       clean       0.00      0.00      0.00        67
       messy       0.98      1.00      0.99      3031

    accuracy                           0.98      3098
   macro avg       0.49      0.50      0.49      3098
weighted avg       0.96      0.98      0.97      3098



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
